In [12]:
import numpy as np
import os 
import glob
import json
from func import *
import ast
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score,roc_curve,auc
import pandas as pd
from sklearn.metrics import roc_auc_score
import shutil
from scipy.stats import sem
from scipy import stats


In [61]:
gt_path = './data/review_result_opt/respiratory'
opt_path = './data/cxr_opt_respiratory'

ais =['without_AI','with_AI']
df = pd.DataFrame(columns = ['name','AI' ,'auc_score'])



for i in range(2,14):
#     if i==13:
#         break
    user1 = 'u'+str(i)
    user2 = 'u'+str(i+12)
    print(user1+'_'+user2)
    
    opt_score = []
    for k in range(len(ais)):
        df_2 = pd.DataFrame(columns = ['name', 'AI', 'gt' ,'opt', 'opt_prob'])
        gt_list = []
        opt_list = []
        opt_prob_list = []
        invisible_cnt = 0
        gt_file = glob.glob(gt_path+'/*.json')
        opt_file = glob.glob(opt_path+'/'+user1+'_'+user2+'/'+ais[k]+'/*.json')
        
        for j in range(len(gt_file)):
            contours_gt, labels, w, h = json_gt_open(gt_file[j])
            contours_opt, contourIds, rating_list, rate = json_opt_open(opt_file[j])
#             print(rating_list,rate)
            # gt        
            if not 'Invisible Nodule' in labels:
                if contours_gt:
                    gt_label = 1
                    gt_list.append(1)
                else:
                    gt_label = 0
                    gt_list.append(0)
                
                # opt
                if contours_opt:
                    opt_label = 1
                    opt_list.append(1)
                    opt_prob = rate/5
                    opt_prob_list.append(rate/5)

                else:
                    opt_label = 0
                    opt_list.append(0)
                    opt_prob = 0
                    opt_prob_list.append(0)
                
            else:
                invisible_cnt+=1
            
            data_label = {'name':user1+'_'+user2, 'AI':ais[k], 'gt': gt_label, 'opt':opt_label, 'opt_prob':opt_prob}
            df_2 = df_2.append(data_label,ignore_index=True)
#             df_2.to_csv('./label_csv/'+user1+'_'+user2+'_'+ais[k]+'.csv')           
        
        opt_score.append(opt_prob_list)    
#         print(invisible_cnt)
        cm = confusion_matrix(gt_list,opt_list)
#         print(cm)
        fp,tp,_=roc_curve(gt_list,opt_prob_list)
#         print('AUC_'+ais[k]+' : ',auc(fp,tp))
        data = {'name':user1+'_'+user2,'AI':ais[k],'auc_score':round(auc(fp,tp),3)}
        df = df.append(data,ignore_index=True)
        
        n_bootstraps = 1000
        rng_seed = 42  # control reproducibility
        bootstrapped_scores = []

        rng = np.random.RandomState(rng_seed)
        for m in range(n_bootstraps):
            y_true = np.array(gt_list)
            y_pred = np.array(opt_prob_list)
            # bootstrap by sampling with replacement on the prediction indices
            indices = rng.randint(0, len(y_pred), len(y_pred))
            if len(np.unique(y_true[indices])) < 2:
                # We need at least one positive and one negative sample for ROC AUC
                # to be defined: reject the sample
                continue

            score = roc_auc_score(y_true[indices], y_pred[indices])
            bootstrapped_scores.append(score)
#             print("Bootstrap #{} ROC area: {:0.3f}".format(m + 1, score))
        
        sorted_scores = np.array(bootstrapped_scores)
        sorted_scores.sort()

        # Computing the lower and upper bound of the 90% confidence interval
        # You can change the bounds percentiles to 0.025 and 0.975 to get
        # a 95% confidence interval instead.
        confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
        confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
        print(round(auc(fp,tp),3), "[{:0.3f} - {:0.3}]".format(confidence_lower, confidence_upper))

u2_u14

0.907 [0.869 - 0.942]

0.905 [0.863 - 0.94]
2
pvalue=0.2646
u3_u15

0.896 [0.854 - 0.936]

0.909 [0.872 - 0.945]
2
pvalue=0.0521
u4_u16

0.853 [0.805 - 0.898]

0.86 [0.814 - 0.904]
2
pvalue=0.5565
u5_u17

0.833 [0.784 - 0.882]

0.879 [0.835 - 0.92]
2
pvalue=0.0212
u6_u18

0.861 [0.813 - 0.907]

0.886 [0.845 - 0.927]
2
pvalue=0.9026
u7_u19

0.829 [0.774 - 0.879]

0.875 [0.828 - 0.917]
2
pvalue=0.0002
u8_u20

0.89 [0.846 - 0.93]

0.914 [0.873 - 0.951]
2
pvalue=0.0147
u9_u21

0.847 [0.799 - 0.892]

0.857 [0.812 - 0.896]
2
pvalue=0.9556
u10_u22

0.859 [0.813 - 0.901]

0.866 [0.820 - 0.907]
2
pvalue=0.8670
u11_u23

0.87 [0.826 - 0.912]

0.879 [0.836 - 0.918]
2
pvalue=0.8076
u12_u24

0.817 [0.771 - 0.859]

0.872 [0.829 - 0.914]
2
pvalue=0.0000
u13_u25

0.854 [0.803 - 0.897]

0.884 [0.842 - 0.926]
2
pvalue=0.7646


In [6]:
df

,name,AI,auc_score
0,u2_u14,without_AI,0.907
1,u2_u14,with_AI,0.905
2,u3_u15,without_AI,0.896
3,u3_u15,with_AI,0.909
4,u4_u16,without_AI,0.853
5,u4_u16,with_AI,0.860
6,u5_u17,without_AI,0.833
7,u5_u17,with_AI,0.879
8,u6_u18,without_AI,0.861
9,u6_u18,with_AI,0.886


In [8]:
df_2.to_excel('lable.xlsx')

In [42]:
gt_path = './data/review_result_opt/healthcheck'
opt_path = './data/cxr_opt_healthcheck'

ais =['without_AI','with_AI']
df_h = pd.DataFrame(columns = ['name','AI' ,'auc_score'])


for i in range(2,11):
#     if i==13:
#         break
    user1 = 'u'+str(i)
    user2 = 'u'+str(i+9)

    print(user1+'_'+user2)
    
    opt_score = []
    for k in range(len(ais)):

        gt_list = []
        opt_list = []
        opt_prob_list = []
        invisible_cnt = 0
        gt_file = glob.glob(gt_path+'/*.json')
        opt_file = glob.glob(opt_path+'/'+user1+'_'+user2+'/'+ais[k]+'/*.json')
        
        for j in range(len(gt_file)):
            contours_gt, labels, w, h = json_gt_open(gt_file[j])
            contours_opt, contourIds, rating_list, rate = json_opt_open(opt_file[j])
#             print(rating_list,rate)
            # gt        
            if not 'Invisible Nodule' in labels:
                if contours_gt:
                    gt_list.append(1)
                else:
                    gt_list.append(0)
                
                # opt
                if contours_opt:
                    opt_list.append(1)
                    opt_prob_list.append(rate/5)

                else:
                    opt_list.append(0)
                    opt_prob_list.append(0)
                
            else:
                invisible_cnt+=1
        
        opt_score.append(opt_prob_list)
#         print(invisible_cnt)
        cm = confusion_matrix(gt_list,opt_list)
#         print(cm)
        fp,tp,_=roc_curve(gt_list,opt_prob_list,pos_label=1)
#         print('AUC_'+ais[k]+' : ',auc(fp,tp))
        data = {'name':user1+'_'+user2,'AI':ais[k],'auc_score':round(auc(fp,tp),3)}
        df_h = df_h.append(data,ignore_index=True)
        
        print(round(auc(fp,tp),3))
        
        n_bootstraps = 1000
        rng_seed = 42  # control reproducibility
        bootstrapped_scores = []

        rng = np.random.RandomState(rng_seed)
        for m in range(n_bootstraps):
            y_true = np.array(gt_list)
            y_pred = np.array(opt_prob_list)
            # bootstrap by sampling with replacement on the prediction indices
            indices = rng.randint(0, len(y_pred), len(y_pred))
            if len(np.unique(y_true[indices])) < 2:
                # We need at least one positive and one negative sample for ROC AUC
                # to be defined: reject the sample
                continue

            score = roc_auc_score(y_true[indices], y_pred[indices])
            bootstrapped_scores.append(score)
#             print("Bootstrap #{} ROC area: {:0.3f}".format(m + 1, score))
        
        sorted_scores = np.array(bootstrapped_scores)
        sorted_scores.sort()

        # Computing the lower and upper bound of the 90% confidence interval
        # You can change the bounds percentiles to 0.025 and 0.975 to get
        # a 95% confidence interval instead.
        confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
        confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
        print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(confidence_lower, confidence_upper))

u2_u11
0.682
Confidence interval for the score: [0.605 - 0.759]
0.695
Confidence interval for the score: [0.621 - 0.775]
u3_u12
0.724
Confidence interval for the score: [0.639 - 0.797]
0.742
Confidence interval for the score: [0.664 - 0.821]
u4_u13
0.754
Confidence interval for the score: [0.667 - 0.836]
0.739
Confidence interval for the score: [0.647 - 0.823]
u5_u14
0.717
Confidence interval for the score: [0.633 - 0.799]
0.726
Confidence interval for the score: [0.641 - 0.81]
u6_u15
0.662
Confidence interval for the score: [0.579 - 0.747]
0.735
Confidence interval for the score: [0.645 - 0.817]
u7_u16
0.704
Confidence interval for the score: [0.626 - 0.781]
0.655
Confidence interval for the score: [0.579 - 0.728]
u8_u17
0.733
Confidence interval for the score: [0.649 - 0.808]
0.726
Confidence interval for the score: [0.632 - 0.809]
u9_u18
0.718
Confidence interval for the score: [0.634 - 0.801]
0.674
Confidence interval for the score: [0.585 - 0.759]
u10_u19
0.635
Confidence interval